In [1]:
from flask import Flask, request, Response, send_from_directory,jsonify
from flask_cors import CORS
from penerjemah import save_and_translate,plance_text
from gogtrans import translate_japanese

import subprocess
import json
import unicodedata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

2023-11-23 22:15:55.739 | INFO     | manga_ocr.ocr:__init__:14 - Loading OCR model from kha-white/manga-ocr-base
2023-11-23 22:16:02.092 | INFO     | manga_ocr.ocr:__init__:26 - Menggunakan CPU
2023-11-23 22:16:02.093 | INFO     | manga_ocr.ocr:__init__:33 - OCR ready


In [2]:
app = Flask(__name__)
CORS(app)

# Menentukan folder tempat file statis disimpan
app.static_folder = 'web'

@app.route("/process_data", methods=["POST"])
def process_data():
    data = request.files['data']  # Ambil data dari permintaan POST untuk text data = request.form['data']    
    bahasa = request.form['bahasa']  
    asal = request.form['asal']
    
    print(asal)
    
     # Periksa apakah file memiliki nama
    if data.filename == '':
        return 'No selected file'
    
    data.save('gambar.png')
    img="gambar.png"
    bjepang,lines = save_and_translate(img)  
    
    print(bjepang)
    #penerjemah disini
    if bahasa=='no':
     translated_text = bjepang
    else:
        print("Menerjemahkan..:")
        translated_text = translate_japanese(bjepang,bahasa,asal)
    
    
    terjemahan=str(translated_text)
    print(translated_text)
    
    # Proses data di sini
    keterangan ="Terjemahkan json ini ke bahasa indonesia, tanpa tanda petik di bagian nomor:"
    processed_data = {key: value.replace('［', '').replace('[', '').replace(']', '') for key, value in bjepang.items()}
    return jsonify({"processed_data": str(processed_data), "keterangan": keterangan,"lines": lines, "terjemahan": terjemahan})

#@app.route("/process_page", methods=["POST"])
#def process_page():
   # page = request.files['data']  # Ambil data dari permintaan POST untuk text data = request.form['data']
    #bahasa = request.form['bahasa']
     # Periksa apakah file memiliki nama
    #if page.filename == '':
        #return 'No selected file'

    #img = PIL.Image.open(page)
    #img.save('gambar.png')
    ##!python -m manga_translator -v --use-cuda --translator=google -l VIN -i gambar.png

    # Buka gambar
    ##imgku = PIL.Image.open('/content/manga-image-translator/result/final.png')

    # Konversi gambar ke format BytesIO
    #img_io = io.BytesIO()
    #imgku.save(img_io, format='PNG')
    #img_io.seek(0)
    #img_data = img_io.read()

    # Konversi gambar ke base64
    #base64_img = base64.b64encode(img_data).decode()

    # Kirim base64_img ke klien web

    #return Response(base64_img, mimetype='text/plain')

@app.route('/style.css')
def style():
    # Menjalankan PHP dan menangkap outputnya
    result = subprocess.run(['php', './web/style.php'], capture_output=True, text=True)

    # Mengembalikan output PHP sebagai respons dengan tipe konten "text/css"
    return Response(result.stdout, content_type='text/css')

@app.route('/')
def final():
     # Menjalankan PHP dan menangkap outputnya
    result = subprocess.run(['php', './web/final.php'], capture_output=True, text=True)

    # Mengembalikan output PHP sebagai respons
    return result.stdout

@app.route('/font/<path:filename>')
def font(filename):
    # Ganti 'web/font' dengan path sesuai dengan struktur folder Anda
    return send_from_directory('web/font/', filename)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Nov/2023 22:16:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2023 22:16:08] "GET /style.css HTTP/1.1" 200 -


ja


Processing images: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.10s/it]


{0: '大変だった日常のカラーのお母', 1: 'もうすぐ着くぞーひゆみ！', 2: '新連載', 3: 'はぁーい', 4: '少女は大きな間違いをしていた', 5: 'やっほーみんな引っ越し先もうすぐでーす', 6: 'ひゆみは「長野県に引っ越し」と聞いただけで――', 7: 'ひゅみの'}
Menerjemahkan..:
{0: '大変だった日常のカラーのお母', 1: 'もうすぐ着くぞーひゆみ！', 2: '新連載', 3: 'はぁーい', 4: '少女は大きな間違いをしていた', 5: 'やっほーみんな引っ越し先もうすぐでーす', 6: 'ひゆみは長野県に引っ越しと聞いただけで――', 7: 'ひゅみの'}


...

Processing images: 100%|███████████████████████████████████████████████████████████████| 12/12 [00:11<00:00,  1.08it/s]


{0: 'っしゃあ！ざまぁ見ろ', 1: '幹部を殺ってやったぜ', 2: 'ふん', 3: '手こずらせやがって', 4: 'まずは．．．一体ですな', 5: 'っづく！', 6: 'うーん', 7: 'これはピンチかも', 8: '．．．', 9: '製品ご利用者と税額の', 10: '３９２', 11: '．．．'}
Menerjemahkan..:
{0: 'っしゃあ！ざまぁ見ろ', 1: '幹部を殺ってやったぜ', 2: 'ふん', 3: '手こずらせやがって', 4: 'まずは．．．一体ですな', 5: 'っづく！', 6: 'うーん', 7: 'これはピンチかも', 8: '．．．', 9: '製品ご利用者と税額の', 10: '３９２', 11: '．．．'}


127.0.0.1 - - [24/Nov/2023 10:20:32] "POST /process_data HTTP/1.1" 200 -


{0: 'Hai!Lihat itu', 1: 'Saya membunuh eksekutif', 2: 'Hmm', 3: 'Tampan', 4: 'Pertama...Itu integral', 5: 'Lakukan!', 6: 'hmmm', 7: 'Ini mungkin keadaan darurat', 8: '...', 9: 'Pengguna produk dan jumlah pajak', 10: '392', 11: '...'}


127.0.0.1 - - [24/Nov/2023 10:21:37] "GET /font/Roboto-Bold.ttf HTTP/1.1" 304 -
127.0.0.1 - - [24/Nov/2023 10:22:07] "GET /font/Super%20Comic.ttf HTTP/1.1" 304 -
127.0.0.1 - - [24/Nov/2023 10:22:07] "GET /font/Next%20Sunday.ttf HTTP/1.1" 304 -
127.0.0.1 - - [24/Nov/2023 10:22:07] "GET /font/BaksoSapi.otf HTTP/1.1" 304 -
127.0.0.1 - - [24/Nov/2023 10:22:07] "GET /font/crimes.ttf HTTP/1.1" 304 -
127.0.0.1 - - [24/Nov/2023 10:22:07] "GET /font/zombie.ttf HTTP/1.1" 304 -
